If you have not already read it, you may want to start with the first tutorial: [Getting started with The Joker](1-Getting-started.ipynb).

# Inferring calibration offsets between instruments

Also in addition to the default linear parameters (see [Tutorial 1](1-Getting-started.ipynb), or the documentation for ``JokerSamples.default()``), *The Joker* allows adding linear parameters to TODO

First, some imports we will need later:

In [ ]:
import astropy.table as at
import astropy.units as u
from astropy.visualization.units import quantity_support
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pymc3 as pm
import exoplanet.units as xu

from thejoker import RVData, JokerPrior, TheJoker, JokerSamples
from thejoker.plot import plot_rv_curves

In [ ]:
data1 = RVData.guess_from_table(at.QTable.read('data-survey1.ecsv'))
data2 = RVData.guess_from_table(at.QTable.read('data-survey2.ecsv'))
data = [data1, data2]

In [ ]:
fig, ax = plt.subplots()
_ = data1.plot(color=None)
_ = data2.plot(color=None)

In [ ]:
with pm.Model() as model:
    dv0_1 = xu.with_unit(pm.Normal('dv0_1', 0, 10),
                         u.km/u.s)
    
    prior = JokerPrior.default(P_min=2*u.day, P_max=256*u.day,
                               sigma_K0=30*u.km/u.s,
                               sigma_v=100*u.km/u.s,
                               v0_offsets=[dv0_1])

prior_samples = prior.sample(size=1_000_000)

In [ ]:
joker = TheJoker(prior)
samples = joker.rejection_sample(data, prior_samples, 
                                 max_posterior_samples=128)
samples

In [ ]:
_ = plot_rv_curves(samples, data=data)

In [ ]:
samples.tbl